# Model details
***Used base model***: bert-base-uncased
***Used dataset***: SQuAD
***Batch size***: 4
***Early stopping patience***: 2
***Early stopping monitor***: val loss
***Datasets***: Train: original 80k train samples, Val: remaining 7599 samples, Test: original 10570 samples
***Loss function***: Sparse categorical crossentropy
***Attempted train epochs***: 10
***Initial learning rate***: 2e-5
***End learning rate***: 0
***Additional info***: Makes use of stride mechanism, the newest evaluation

In [1]:
from transformers import (
    AutoTokenizer,
    TFAutoModelForQuestionAnswering,
    DefaultDataCollator,
)
import tensorflow as tf
from question_answering.constants import constants
from question_answering.utils import core_qa_utils
from question_answering.utils.extractive_qa import (
    model_management,
    predictions,
    squad_metrics,
    pure_metrics,
)
from question_answering.paths import extractive_qa_paths
import ipynbname
from question_answering.keras_callbacks.time_measure_callback import TimeMeasureCallback
import numpy as np
from datasets import Dataset

# Data loading

In [2]:
raw_train_dataset, raw_test_dataset = core_qa_utils.load_datasets_from_json(
    dataset_path=extractive_qa_paths.squad_dataset_dir,
    filenames=["original_train.json", "original_test.json"],
)

In [3]:
train_dataset = raw_train_dataset.select(range(80000))
val_dataset = raw_train_dataset.select(range(80000, 87599))
test_dataset = raw_test_dataset

In [4]:
train_dataset, val_dataset, test_dataset

(Dataset({
     features: ['id', 'title', 'context', 'question', 'answers', 'answer_text', 'answer_start'],
     num_rows: 80000
 }),
 Dataset({
     features: ['id', 'title', 'context', 'question', 'answers', 'answer_text', 'answer_start'],
     num_rows: 7599
 }),
 Dataset({
     features: ['id', 'title', 'context', 'question', 'answers', 'answer_text', 'answer_start'],
     num_rows: 10570
 }))

# Data preprocessing

In [5]:
model_checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [6]:
max_length = 384
stride = 128

In [7]:
def preprocess_training_dataset(dataset):
    questions = [q.strip() for q in dataset["question"]]
    contexts = [c.strip() for c in dataset["context"]]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        padding="max_length",
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answer_starts = dataset["answer_start"]
    answer_texts = dataset["answer_text"]
    start_positions = []
    end_positions = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        answer_start = answer_starts[sample_idx][0]
        answer_text = answer_texts[sample_idx][0]
        start_char = answer_start
        end_char = start_char + len(answer_text)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end TOKEN positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

In [8]:
def preprocess_test_samples(dataset):
    questions = [q.strip() for q in dataset["question"]]
    contexts = [c.strip() for c in dataset["context"]]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        padding="max_length",
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answer_starts_batch = dataset["answer_start"]
    answer_texts_batch = dataset["answer_text"]
    start_positions = []
    end_positions = []
    example_ids = []
    new_offset_mapping = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        example_id = dataset["id"][sample_idx]
        answer_starts = answer_starts_batch[sample_idx]
        answer_texts = answer_texts_batch[sample_idx]
        sequence_ids = inputs.sequence_ids(i)
        sample_start_positions = []
        sample_end_positions = []

        for j, answer_start in enumerate(answer_starts):
            answer_text = answer_texts[j]
            start_char = answer_start
            end_char = start_char + len(answer_text)

            # Find the start and end of the context
            idx = 0
            while sequence_ids[idx] != 1:
                idx += 1
            context_start = idx
            while sequence_ids[idx] == 1:
                idx += 1
            context_end = idx - 1

            # If the answer is not fully inside the context, label is (0, 0)
            if (
                offset[context_start][0] > start_char
                or offset[context_end][1] < end_char
            ):
                sample_start_positions.append(0)
                sample_end_positions.append(0)
            else:
                # Otherwise it's the start and end TOKEN positions
                idx = context_start
                while idx <= context_end and offset[idx][0] <= start_char:
                    idx += 1
                sample_start_positions.append(idx - 1)

                idx = context_end
                while idx >= context_start and offset[idx][1] >= end_char:
                    idx -= 1
                sample_end_positions.append(idx + 1)

        new_offset_mapping.append(
            [o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)]
        )

        start_positions.append(sample_start_positions)
        end_positions.append(sample_end_positions)
        example_ids.append(example_id)

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs["example_id"] = example_ids
    inputs["offset_mapping"] = new_offset_mapping
    return inputs

In [9]:
tokenized_train_dataset = train_dataset.map(
    preprocess_training_dataset, batched=True, remove_columns=train_dataset.column_names
)
tokenized_val_dataset = val_dataset.map(
    preprocess_training_dataset, batched=True, remove_columns=val_dataset.column_names
)

In [10]:
tokenized_test_dataset = test_dataset.map(
    preprocess_test_samples, batched=True, remove_columns=test_dataset.column_names
)

# Training preparation

In [11]:
# Model parameters
full_model_name = "-".join(ipynbname.name().split("_"))

# Checkpoints
checkpoint_filename_template = constants.checkpoint_filename_template
checkpoints_path = (
    extractive_qa_paths.training_checkpoints_dir
    / full_model_name
    / checkpoint_filename_template
)

# Saved models
saved_models_path = extractive_qa_paths.saved_models_dir / full_model_name

# Evaluation
model_evaluation_dir = extractive_qa_paths.model_evaluation_dir / full_model_name
figures_dir = model_evaluation_dir / constants.figures_dir_name

# Hyperparameters
batch_size = 4
train_epochs = 10
initial_learning_rate = 2e-5
end_learning_rate = 0

In [12]:
data_collator = DefaultDataCollator(return_tensors="tf")

tf_train_dataset = core_qa_utils.convert_to_tf_dataset(
    hf_dataset=tokenized_train_dataset,
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["start_positions", "end_positions"],
    collator=data_collator,
    batch_size=batch_size,
)
tf_val_dataset = core_qa_utils.convert_to_tf_dataset(
    hf_dataset=tokenized_val_dataset,
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=["start_positions", "end_positions"],
    collator=data_collator,
    batch_size=batch_size,
)
tf_test_dataset = core_qa_utils.convert_to_tf_dataset(
    hf_dataset=tokenized_test_dataset,
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=None,
    collator=data_collator,
    batch_size=batch_size,
)

In [13]:
model = TFAutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
    checkpoints_path, verbose=1, save_weights_only=True
)
early_stop_cb = tf.keras.callbacks.EarlyStopping(patience=2)
time_measure_cb = TimeMeasureCallback()

callbacks = [checkpoint_cb, early_stop_cb, time_measure_cb]

In [15]:
num_train_steps = len(tf_train_dataset) * train_epochs

lr_scheduler = tf.keras.optimizers.schedules.PolynomialDecay(
    initial_learning_rate=initial_learning_rate,
    end_learning_rate=end_learning_rate,
    decay_steps=num_train_steps,
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_scheduler)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
qa_metrics = ["accuracy"]

model.compile(optimizer=optimizer, loss=loss, metrics=qa_metrics)

In [16]:
tf.keras.mixed_precision.set_global_policy("mixed_float16")

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Ti, compute capability 8.6


# Model training

In [ ]:
history = model.fit(
    tf_train_dataset,
    validation_data=tf_val_dataset,
    epochs=train_epochs,
    callbacks=callbacks,
)

In [ ]:
history = history.history

In [ ]:
core_qa_utils.save_dict_as_json(
    dictionary=history, dir_path=model_evaluation_dir, filename="history.json"
)

In [ ]:
# Plot training figures
core_qa_utils.plot_and_save_fig_from_history(
    history,
    attributes=["start_logits_accuracy", "val_start_logits_accuracy"],
    title="Model start logits accuracy",
    y_label="Accuracy",
    x_label="Epoch",
    legend_descriptors=["Train", "Val"],
    figure_dir_path=figures_dir,
    figure_filename=f"{full_model_name}_start_accuracy.png",
)

core_qa_utils.plot_and_save_fig_from_history(
    history,
    attributes=["end_logits_accuracy", "val_end_logits_accuracy"],
    title="Model end logits accuracy",
    y_label="Accuracy",
    x_label="Epoch",
    legend_descriptors=["Train", "Val"],
    figure_dir_path=figures_dir,
    figure_filename=f"{full_model_name}_end_accuracy.png",
)

core_qa_utils.plot_and_save_fig_from_history(
    history,
    attributes=["loss", "val_loss"],
    title="Model loss",
    y_label="Loss",
    x_label="Epoch",
    legend_descriptors=["Train", "Val"],
    figure_dir_path=figures_dir,
    figure_filename=f"{full_model_name}_loss.png",
)

core_qa_utils.plot_and_save_fig_from_history(
    history,
    attributes=["start_logits_loss", "val_start_logits_loss"],
    title="Model start logits loss",
    y_label="Loss",
    x_label="Epoch",
    legend_descriptors=["Train", "Val"],
    figure_dir_path=figures_dir,
    figure_filename=f"{full_model_name}_start_loss.png",
)

core_qa_utils.plot_and_save_fig_from_history(
    history,
    attributes=["end_logits_loss", "val_end_logits_loss"],
    title="Model end logits loss",
    y_label="Loss",
    x_label="Epoch",
    legend_descriptors=["Train", "Val"],
    figure_dir_path=figures_dir,
    figure_filename=f"{full_model_name}_end_loss.png",
)

# Saving model's best version & training data

In [ ]:
history = core_qa_utils.read_json_as_dict(model_evaluation_dir / "history.json")

In [ ]:
best_epoch = core_qa_utils.get_best_epoch(
    history=history, metric="val_loss", metric_evaluator="min"
)

In [ ]:
training_data = {
    "history": history,
    "attempted_epochs": train_epochs,
    "best_epoch": best_epoch,
    "training_time": time_measure_cb.total_training_time(),
    "gpu": core_qa_utils.get_gpu_name(),
}

In [ ]:
core_qa_utils.save_dict_as_json(
    training_data, dir_path=model_evaluation_dir, filename="training_data.json"
)

In [ ]:
best_model = model_management.load_best_model_from_checkpoints(
    model=model, model_name=full_model_name, epoch=best_epoch, remove_checkpoints=True
)

In [ ]:
model_management.save_model(model=best_model, model_name=full_model_name)

# Getting predictions

In [23]:
loaded_model = model_management.load_model(
    model_checkpoint=model_checkpoint, model_name=full_model_name
)
loaded_model.compile(optimizer=optimizer, loss=loss, metrics=qa_metrics)

All PyTorch model weights were used when initializing TFBertForQuestionAnswering.

Some weights or buffers of the TF 2.0 model TFBertForQuestionAnswering were not initialized from the PyTorch model and are newly initialized: ['qa_outputs.weight', 'qa_outputs.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
output = loaded_model.predict(tf_test_dataset)

2696/2696 [==============================] - 121s 43ms/step


In [19]:
# Get predictions
output_start_logits = predictions.get_preds(
    output, output_key="start_logits", return_type="logits"
)
output_end_logits = predictions.get_preds(
    output, output_key="end_logits", return_type="logits"
)

output_start_positions = predictions.get_preds(
    output, output_key="start_logits", return_type="class"
)
output_end_positions = predictions.get_preds(
    output, output_key="end_logits", return_type="class"
)

In [27]:
def get_predicted_texts_for_multiple_n_best(
    start_logits: np.ndarray,
    end_logits: np.ndarray,
    features: Dataset,
    examples: Dataset,
    n_best_variants: list[int],
):
    predicted_texts_for_varying_n_best = []
    for n_best_variant in n_best_variants:
        predicted_texts = predictions.get_predicted_texts(
            start_logits=start_logits,
            end_logits=end_logits,
            features=features,
            examples=examples,
            n_best=n_best_variant,
        )
        predicted_texts_for_varying_n_best.append(predicted_texts)

    return predicted_texts_for_varying_n_best

In [21]:
n_bests = ["01", "02", "03", "05", "10", "20"]

textual_predictions_variants = get_predicted_texts_for_multiple_n_best(
    start_logits=output_start_logits,
    end_logits=output_end_logits,
    features=tokenized_test_dataset,
    examples=test_dataset,
    n_best_variants=[int(n_best) for n_best in n_bests],
)

# Metrics & evaluation

### Accuracies

In [22]:
accuracies_metrics = squad_metrics.calculate_squad_accuracies(
    start_actual=tokenized_test_dataset["start_positions"],
    end_actual=tokenized_test_dataset["end_positions"],
    start_preds=output_start_positions,
    end_preds=output_end_positions,
)
accuracies_metrics

{'start_accuracy': 0.8177856083086054,
 'end_accuracy': 0.8370734421364985,
 'full_accuracy': 0.7366468842729971}

### Precision, recall, F1-score, exact match

In [28]:
def calculate_all_variants_of_squad_qa_metrics(
    answers: list[list[str]],
    predicted_texts_variants: list[list[str]],
):
    metrics = []
    for predicted_texts_variant in predicted_texts_variants:
        non_normalized_metrics = squad_metrics.calculate_squad_qa_metrics(
            answers=answers, predicted_texts=predicted_texts_variant, normalize=False
        )
        normalized_metrics = squad_metrics.calculate_squad_qa_metrics(
            answers=answers, predicted_texts=predicted_texts_variant, normalize=True
        )
        metrics.append(non_normalized_metrics)
        metrics.append(normalized_metrics)

    return metrics

In [ ]:
qa_metrics = calculate_all_variants_of_squad_qa_metrics(
    answers=test_dataset["answer_text"],
    predicted_texts_variants=textual_predictions_variants,
)

### Correct predictions by sentence lengths

In [ ]:
# # Plot correctness of the model by sentence length
# core_qa_utils.plot_correct_predictions_by_sentence_length(
#     sentences=model_predictions["context"],
#     correctly_predicted=model_predictions["correctly_predicted_lowercase"],
#     figure_path=figures_dir
#                 / f"{full_model_name}_context_sentence_lengths_to_correct_preds_0_300_50.png",
#     figure_title="Correctness by sentence length",
#     divider=50,
#     min_threshold=0,
#     max_threshold=300,
# )
#
# core_qa_utils.plot_correct_predictions_by_sentence_length(
#     sentences=model_predictions["context"],
#     correctly_predicted=model_predictions["correctly_predicted_lowercase"],
#     figure_path=figures_dir
#                 / f"{full_model_name}_context_sentence_lengths_to_correct_preds_0_400_80.png",
#     figure_title="Correctness by sentence length",
#     divider=80,
#     min_threshold=0,
#     max_threshold=400,
# )
#
# core_qa_utils.plot_correct_predictions_by_sentence_length(
#     sentences=model_predictions["context"],
#     correctly_predicted=model_predictions["correctly_predicted_lowercase"],
#     figure_path=figures_dir
#                 / f"{full_model_name}_context_sentence_lengths_to_correct_preds_0_400_100.png",
#     figure_title="Correctness by sentence length",
#     divider=100,
#     min_threshold=0,
#     max_threshold=400,
# )
#
# core_qa_utils.plot_correct_predictions_by_sentence_length(
#     sentences=model_predictions["context"],
#     correctly_predicted=model_predictions["correctly_predicted_lowercase"],
#     figure_path=figures_dir
#                 / f"{full_model_name}_context_sentence_lengths_to_correct_preds_30_180_30.png",
#     figure_title="Correctness by sentence length",
#     divider=30,
#     min_threshold=30,
#     max_threshold=180,
# )

### Saving evaluation data

In [26]:
evaluation_data = {
    "accuracy": accuracies_metrics,
    "precision": {
        f"{n_bests[i // 2]}_best_{'normalized' if i % 2 else 'standard'}": qa_metrics[
            i
        ]["precision"]
        for i in range(len(n_bests) * 2)
    },
    "recall": {
        f"{n_bests[i // 2]}_best_{'normalized' if i % 2 else 'standard'}": qa_metrics[
            i
        ]["recall"]
        for i in range(len(n_bests) * 2)
    },
    "f1": {
        f"{n_bests[i // 2]}_best_{'normalized' if i % 2 else 'standard'}": qa_metrics[
            i
        ]["f1"]
        for i in range(len(n_bests) * 2)
    },
    "exact_match": {
        f"{n_bests[i // 2]}_best_{'normalized' if i % 2 else 'standard'}": qa_metrics[
            i
        ]["exact_match"]
        for i in range(len(n_bests) * 2)
    },
}

In [27]:
core_qa_utils.save_dict_as_json(
    evaluation_data, dir_path=model_evaluation_dir, filename="evaluation_data.json"
)

# Additional analysis

##### 1. How many of bad predictions have start pred greater than end pred?
- Retrieve bad preds
- How many have start pred greater than end pred?
- How many of these can be turned into good predictions?

##### 2. Could check if accuracy == exact match, when using same amounts of samples for calculating these metrics

##### 3. Take a deeper look into predictions and ground truth - especially at the normalization and the way it improves the results
- Get predictions where normalized result is good but normal one isn't
- In case of correct normal predictions, are normalized ones always good? - Is it always an improvement?

In [28]:
# T. B. D.

# Medical dataset evaluation

In [57]:
raw_medical_dataset = core_qa_utils.load_datasets_from_json(
    dataset_path=extractive_qa_paths.medical_dataset_dir,
    filenames=["original_medical.json"],
)[0]

In [58]:
def get_context_span(context: str, answer_start: int, answer_text: str, window: int):
    # Define the start and end indices for the context span
    start_span = max(0, answer_start - window)
    end_span = min(len(context), answer_start + len(answer_text) + window)

    # Extract the context span containing the answer
    context_span = context[start_span:end_span]

    return {"context_span": context_span, "answer_start_shift": start_span}

In [59]:
window_size = 750

In [70]:
def span_dataset(dataset):
    answer_starts = dataset["answer_start"]
    answer_texts = dataset["answer_text"]
    contexts = dataset["context"]

    for i, context in enumerate(contexts):
        context_span_result = get_context_span(
            context=context,
            answer_start=answer_starts[i][0],
            answer_text=answer_texts[i][0],
            window=window_size,
        )

        contexts[i] = context_span_result["context_span"]
        answer_starts[i] = [
            answer_starts[i][0] - context_span_result["answer_start_shift"]
        ]

    dataset["context"] = contexts
    dataset["answer_start"] = answer_starts
    return dataset

In [71]:
spanned_medical_dataset = raw_medical_dataset.map(span_dataset, batched=True)

Map:   0%|          | 0/2019 [00:00<?, ? examples/s]

In [72]:
def preprocess_medical_dataset(dataset):
    questions = [q.strip() for q in dataset["question"]]
    contexts = [c.strip() for c in dataset["context"]]

    inputs = tokenizer(
        questions,
        contexts,
        max_length=max_length,
        padding="max_length",
        truncation="only_second",
        stride=stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
    )

    offset_mapping = inputs.pop("offset_mapping")
    sample_map = inputs.pop("overflow_to_sample_mapping")
    answer_starts = dataset["answer_start"]
    answer_texts = dataset["answer_text"]
    start_positions = []
    end_positions = []
    example_ids = []
    new_offset_mapping = []

    for i, offset in enumerate(offset_mapping):
        sample_idx = sample_map[i]
        example_id = dataset["id"][sample_idx]
        answer_start = answer_starts[sample_idx][0]
        answer_text = answer_texts[sample_idx][0]
        start_char = answer_start
        end_char = start_char + len(answer_text)
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        idx = 0
        while sequence_ids[idx] != 1:
            idx += 1
        context_start = idx
        while sequence_ids[idx] == 1:
            idx += 1
        context_end = idx - 1

        # If the answer is not fully inside the context, label is (0, 0)
        if offset[context_start][0] > start_char or offset[context_end][1] < end_char:
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise it's the start and end TOKEN positions
            idx = context_start
            while idx <= context_end and offset[idx][0] <= start_char:
                idx += 1
            start_positions.append(idx - 1)

            idx = context_end
            while idx >= context_start and offset[idx][1] >= end_char:
                idx -= 1
            end_positions.append(idx + 1)

        example_ids.append(example_id)
        new_offset_mapping.append(
            [o if sequence_ids[k] == 1 else None for k, o in enumerate(offset)]
        )

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    inputs["example_id"] = example_ids
    inputs["offset_mapping"] = new_offset_mapping
    return inputs

In [109]:
tokenized_medical_dataset = spanned_medical_dataset.map(
    preprocess_medical_dataset,
    batched=True,
    remove_columns=spanned_medical_dataset.column_names,
)

In [75]:
tf_medical_dataset = core_qa_utils.convert_to_tf_dataset(
    hf_dataset=tokenized_medical_dataset,
    columns=["input_ids", "token_type_ids", "attention_mask"],
    label_cols=None,
    collator=data_collator,
    batch_size=batch_size,
)

In [76]:
medical_output = loaded_model.predict(tf_medical_dataset)

708/708 [==============================] - 30s 43ms/step


In [77]:
# Get predictions
medical_output_start_logits = predictions.get_preds(
    medical_output, output_key="start_logits", return_type="logits"
)
medical_output_end_logits = predictions.get_preds(
    medical_output, output_key="end_logits", return_type="logits"
)

medical_output_start_positions = predictions.get_preds(
    medical_output, output_key="start_logits", return_type="class"
)
medical_output_end_positions = predictions.get_preds(
    medical_output, output_key="end_logits", return_type="class"
)

In [90]:
n_bests = ["01", "02", "03", "05", "10", "20"]

medical_textual_predictions_variants = get_predicted_texts_for_multiple_n_best(
    start_logits=medical_output_start_logits,
    end_logits=medical_output_end_logits,
    features=tokenized_medical_dataset,
    examples=spanned_medical_dataset,
    n_best_variants=[int(n_best) for n_best in n_bests],
)

# Metrics & evaluation

### Accuracies

In [111]:
medical_accuracies_metrics = pure_metrics.calculate_pure_accuracies(
    start_actual=tokenized_medical_dataset["start_positions"],
    end_actual=tokenized_medical_dataset["end_positions"],
    start_preds=medical_output_start_positions,
    end_preds=medical_output_end_positions,
)
medical_accuracies_metrics

{'start_accuracy': 0.4344754503708937,
 'end_accuracy': 0.3924408336276934,
 'full_accuracy': 0.3306252207700459}

### Precision, recall, F1-score, exact match

In [92]:
def calculate_all_variants_of_pure_qa_metrics(
    answers: list[str],
    predicted_texts_variants: list[list[str]],
):
    metrics = []
    for predicted_texts_variant in predicted_texts_variants:
        non_normalized_metrics = pure_metrics.calculate_pure_qa_metrics(
            answers=answers, predicted_texts=predicted_texts_variant, normalize=False
        )
        normalized_metrics = pure_metrics.calculate_pure_qa_metrics(
            answers=answers, predicted_texts=predicted_texts_variant, normalize=True
        )
        metrics.append(non_normalized_metrics)
        metrics.append(normalized_metrics)

    return metrics

In [95]:
valid_answers = [sample["answer_text"][0] for sample in spanned_medical_dataset]
medical_qa_metrics = calculate_all_variants_of_pure_qa_metrics(
    answers=valid_answers,
    predicted_texts_variants=medical_textual_predictions_variants,
)

In [122]:
qa_metrics[7]

{'precision': 0.7322483275975379,
 'recall': 0.47246112438361215,
 'f1': 0.5129497342916799,
 'exact_match': 0.24962852897473997}

In [ ]:
# # Plot correctness of the model by sentence length
# core_qa_utils.plot_correct_predictions_by_sentence_length(
#     sentences=model_predictions["context"],
#     correctly_predicted=model_predictions["correctly_predicted_lowercase"],
#     figure_path=figures_dir
#                 / f"{full_model_name}_context_sentence_lengths_to_correct_preds_0_300_50.png",
#     figure_title="Correctness by sentence length",
#     divider=50,
#     min_threshold=0,
#     max_threshold=300,
# )
#
# core_qa_utils.plot_correct_predictions_by_sentence_length(
#     sentences=model_predictions["context"],
#     correctly_predicted=model_predictions["correctly_predicted_lowercase"],
#     figure_path=figures_dir
#                 / f"{full_model_name}_context_sentence_lengths_to_correct_preds_0_400_80.png",
#     figure_title="Correctness by sentence length",
#     divider=80,
#     min_threshold=0,
#     max_threshold=400,
# )
#
# core_qa_utils.plot_correct_predictions_by_sentence_length(
#     sentences=model_predictions["context"],
#     correctly_predicted=model_predictions["correctly_predicted_lowercase"],
#     figure_path=figures_dir
#                 / f"{full_model_name}_context_sentence_lengths_to_correct_preds_0_400_100.png",
#     figure_title="Correctness by sentence length",
#     divider=100,
#     min_threshold=0,
#     max_threshold=400,
# )
#
# core_qa_utils.plot_correct_predictions_by_sentence_length(
#     sentences=model_predictions["context"],
#     correctly_predicted=model_predictions["correctly_predicted_lowercase"],
#     figure_path=figures_dir
#                 / f"{full_model_name}_context_sentence_lengths_to_correct_preds_30_180_30.png",
#     figure_title="Correctness by sentence length",
#     divider=30,
#     min_threshold=30,
#     max_threshold=180,
# )

### Saving evaluation data

In [26]:
evaluation_data = {
    "accuracy": accuracies_metrics,
    "precision": {
        f"{n_bests[i // 2]}_best_{'normalized' if i % 2 else 'standard'}": qa_metrics[
            i
        ]["precision"]
        for i in range(len(n_bests) * 2)
    },
    "recall": {
        f"{n_bests[i // 2]}_best_{'normalized' if i % 2 else 'standard'}": qa_metrics[
            i
        ]["recall"]
        for i in range(len(n_bests) * 2)
    },
    "f1": {
        f"{n_bests[i // 2]}_best_{'normalized' if i % 2 else 'standard'}": qa_metrics[
            i
        ]["f1"]
        for i in range(len(n_bests) * 2)
    },
    "exact_match": {
        f"{n_bests[i // 2]}_best_{'normalized' if i % 2 else 'standard'}": qa_metrics[
            i
        ]["exact_match"]
        for i in range(len(n_bests) * 2)
    },
}

In [27]:
core_qa_utils.save_dict_as_json(
    evaluation_data, dir_path=model_evaluation_dir, filename="evaluation_data.json"
)

# Saving complete results

In [28]:
training_data = core_qa_utils.read_json_as_dict(
    model_evaluation_dir / "training_data.json"
)

In [29]:
evaluation_data = core_qa_utils.read_json_as_dict(
    model_evaluation_dir / "evaluation_data.json"
)

In [30]:
complete_evaluation_data = {"training": training_data, "evaluation": evaluation_data}

In [31]:
core_qa_utils.save_dict_as_json(
    complete_evaluation_data,
    dir_path=model_evaluation_dir,
    filename="complete_evaluation_data.json",
)